In [29]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
# from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
music_three_sec_data = pd.read_csv('Data/features_3_sec.csv')
music_three_sec_data.head(10)

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues
5,blues.00000.5.wav,66149,0.376670,0.089702,0.132618,0.003583,1994.915219,211700.619569,2152.767854,74263.873102,...,31.448069,-3.448373,34.284130,-0.416165,40.791092,-3.649625,32.457901,3.025218,28.892687,blues
6,blues.00000.6.wav,66149,0.379909,0.088827,0.130335,0.003166,1962.150096,177443.070045,2146.503479,98020.541422,...,33.954071,-2.068194,25.623655,1.428141,47.957699,-3.267124,39.382240,3.276939,25.999132,blues
7,blues.00000.7.wav,66149,0.331880,0.092119,0.140600,0.002546,1701.890924,35678.130616,1979.387612,36670.725886,...,38.456211,-3.637886,24.530296,-0.105148,26.716150,-2.016985,23.150423,0.210787,42.512966,blues
8,blues.00000.8.wav,66149,0.347877,0.094209,0.133130,0.002538,1746.473502,138073.931244,1887.619723,117069.920049,...,44.311455,-4.370029,29.873167,2.114592,33.843155,-2.264663,80.812393,3.758598,97.618835,blues
9,blues.00000.9.wav,66149,0.358061,0.082957,0.115312,0.001846,1763.948942,61493.423121,1874.195710,51944.921435,...,43.967834,-3.448304,48.671944,0.099792,41.839546,-7.677177,96.253654,0.791776,40.416420,blues


In [3]:
music_three_sec_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9990 entries, 0 to 9989
Data columns (total 60 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   filename                 9990 non-null   object 
 1   length                   9990 non-null   int64  
 2   chroma_stft_mean         9990 non-null   float64
 3   chroma_stft_var          9990 non-null   float64
 4   rms_mean                 9990 non-null   float64
 5   rms_var                  9990 non-null   float64
 6   spectral_centroid_mean   9990 non-null   float64
 7   spectral_centroid_var    9990 non-null   float64
 8   spectral_bandwidth_mean  9990 non-null   float64
 9   spectral_bandwidth_var   9990 non-null   float64
 10  rolloff_mean             9990 non-null   float64
 11  rolloff_var              9990 non-null   float64
 12  zero_crossing_rate_mean  9990 non-null   float64
 13  zero_crossing_rate_var   9990 non-null   float64
 14  harmony_mean            

In [5]:
X = music_three_sec_data.iloc[:,1:-1]
Y = music_three_sec_data.label

In [10]:
X_np = X.to_numpy()
print(X_np.shape)

(9990, 58)


In [14]:
X_ts = []
Y_ts = []
window_size = 50

for i in range(X_np.shape[0] - window_size):
    X_ts.append(X_np[i: window_size + i, :].tolist())
    Y_ts.append(Y[i])

X_ts_np = np.array(X_ts, dtype=object).astype('float32')
Y_ts_np = np.array(Y_ts, dtype=object)

print(X_ts_np.shape)
print(Y_ts_np.shape)

(9940, 50, 58)
(9940,)


In [17]:
label_encoder = LabelEncoder()
Y_ts_np = label_encoder.fit_transform(Y_ts_np)

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X_ts_np, Y_ts_np, test_size=0.2, stratify=Y_ts_np, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, stratify=Y_train, random_state=42)

In [41]:
def baseline_model():
    input_shape = X_train.shape[1:]
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(64))
    model.add(Dense(64, activation="relu"))
    model.add(Dense(10, activation="softmax"))
    
    # optimizer = Adam(lr=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [52]:
bn_model = baseline_model()

bn_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=60, batch_size=32, verbose=2)

Epoch 1/60
199/199 - 45s - loss: 1.7612 - accuracy: 0.3503 - val_loss: 1.5093 - val_accuracy: 0.4161 - 45s/epoch - 224ms/step
Epoch 2/60
199/199 - 17s - loss: 1.4996 - accuracy: 0.4131 - val_loss: 1.3989 - val_accuracy: 0.4557 - 17s/epoch - 86ms/step
Epoch 3/60
199/199 - 16s - loss: 1.4729 - accuracy: 0.4350 - val_loss: 1.3542 - val_accuracy: 0.4714 - 16s/epoch - 81ms/step
Epoch 4/60
199/199 - 17s - loss: 1.5067 - accuracy: 0.4139 - val_loss: 1.6924 - val_accuracy: 0.3652 - 17s/epoch - 85ms/step
Epoch 5/60
199/199 - 17s - loss: 1.5222 - accuracy: 0.4080 - val_loss: 1.4796 - val_accuracy: 0.4098 - 17s/epoch - 85ms/step
Epoch 6/60
199/199 - 16s - loss: 1.4856 - accuracy: 0.4212 - val_loss: 1.4958 - val_accuracy: 0.4192 - 16s/epoch - 82ms/step
Epoch 7/60
199/199 - 17s - loss: 1.4615 - accuracy: 0.4267 - val_loss: 1.4464 - val_accuracy: 0.4387 - 17s/epoch - 86ms/step
Epoch 8/60
199/199 - 16s - loss: 1.4329 - accuracy: 0.4353 - val_loss: 1.3691 - val_accuracy: 0.4645 - 16s/epoch - 81ms/step

In [53]:
bn_model.save("music_LSTM.h5")

In [54]:
validation_accuracy = bn_model.evaluate(X_val, Y_val)
training_accuracy = bn_model.evaluate(X_train, Y_train)
testing_accuracy = bn_model.evaluate(X_test, Y_test)
# print of test error used only after development of the model
print("\nTraining accuracy: %f\t Validation accuracy: %f\t Testing Accuracy: %f" % (training_accuracy[1], validation_accuracy[1], testing_accuracy[1]))
print("\nTraining loss: %f    \t Validation loss: %f    \t Testing Loss: %f \n" % (training_accuracy[0], validation_accuracy[0], testing_accuracy[0]))
